In [ ]:
# -*- coding: utf-8 -*-
import sys
import os
import pandas as pd
import xml.etree.ElementTree as ET
import numpy as np
import xlrd
import openpyxl

sys.path.append("..")
from config import *
from lib import *

In [ ]:
# xdf = ldf.join(rdf)
file = merge_result_output_filename
file = os.path.join(TEMP_DIR, 'Tyranny_tempCustom.xlsx')
xdf = pd.read_excel(file, index_col='index').fillna('')

In [ ]:
# 提取匹配字符
re_url = '(\[url*?[^\]]*\].*?\[/url\])'
re_gloss = '(\[url=glossary:(.*?[^\]]*)\].*?\[/url\])'

xdf['DefaultText'].str.extract(re_gloss,expand=True).drop_duplicates().fillna('')

xdf['[ali]DefaultText'].str.extract(re_gloss,expand=True).drop_duplicates().fillna('')

In [ ]:
ldf = xdf['DefaultText'].str.extract(re_gloss,expand=True).drop_duplicates(1).fillna('')
rdf = xdf['merged'].str.extract(re_gloss,expand=True).drop_duplicates().fillna('')

ind = ldf.join(rdf, how='outer',rsuffix='_m')
ind.columns = ['Eng', 'e1', 'm', 'm1']

ind
ind.to_excel(os.path.join(TEMP_DIR,'glossary_rep.xlsx'), index=False)

In [218]:
# {0}
re_var = "\{*\}"
vas = xdf[xdf['DefaultText'].str.contains(re_var)]

In [219]:
# no_trans
def not_ch(text):
    return not has_ch(text) and text !=''
nots = xdf[xdf['merged'].apply(not_ch)].drop_duplicates()

In [220]:
ha = pd.merge(vas, nots, how='outer')
fixdf = pd.read_excel(os.path.join(STORAGE_DIR,'Tyr_guifix.xlsx'), index_col='index').fillna('').drop_duplicates()

In [221]:
pdf = pd.merge(ha, fixdf, how='outer').fillna('')
pdf.index = [pdf['Name'] + '_' + pdf['ID'].astype('str')]
# pdf = pdf.sort(['Name','ID'])

In [222]:
pdf[['Name','ID','DefaultText','merged','Fixed','说明']].to_excel(os.path.join(TEMP_DIR, 'neo.xlsx'))

In [ ]:
quo = xdf['DefaultText'].str.extract('(\[[^url/url][^\]]*\])',expand=True).drop_duplicates().fillna('')

In [ ]:
quo.to_excel(os.path.join(TEMP_DIR, 'quo.xlsx'))

In [ ]:
# e.g 
# [url=glossary:blinded]Blinded[/url] keyilo [url=glossary:blinded]目盲[/url]

# \[url=glossary:blinded.*?(?=\])]
# >> [url=glossary:blinded]

# \[(url*?)[^\]]*\].*?\[/url\]
# >> [url=glossary:blinded]Blinded[/url] || [url=glossary:blinded]目盲[/url]

gloss = os.path.join(STORAGE_DIR, 'tyr_gloss.xlsx')
gl = pd.read_excel(gloss)
gl = gl.join(gl[0].str.extract('(\[url=(.*?[^\]]*)\].*?\[/url\])',expand=True)[1], how='outer')
gl